In [1]:
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor

def start_stable_driver():
    options = Options()
    options.page_load_strategy = 'eager'
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("detach", True)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    prefs = {"profile.managed_default_content_settings.images": 2} # Resimleri kapat
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

# Global Ayarlar
BASE_URL = "https://www.sikayetvar.com/arcelik/klima"
klima_links = [] # Klima link havuzu

In [2]:
target_pages = 200 
current_page = 1 
refresh_limit = 20 

print(f"🚀 Arçelik Klima linkleri toplanıyor. Hedef: {target_pages} sayfa.")

while current_page <= target_pages:
    driver = start_stable_driver()
    next_stop = min(current_page + refresh_limit - 1, target_pages)
    print(f"🌐 Yeni Oturum Başladı: Sayfa {current_page} - {next_stop}")
    
    try:
        for p in range(current_page, next_stop + 1):
            success = False
            attempts = 0
            while not success and attempts < 2:
                try:
                    driver.get(f"{BASE_URL}?page={p}")
                    time.sleep(2.5)
                    
                    links = driver.find_elements(By.CSS_SELECTOR, "a.complaint-layer")
                    if len(links) > 0:
                        for l in links:
                            href = l.get_attribute("href")
                            if href and "/arcelik/" in href:
                                if not any(d['url'] == href for d in klima_links):
                                    klima_links.append({"baslik": l.get_attribute("title"), "url": href})
                        
                        if p % 5 == 0:
                            print(f"✅ Sayfa {p} bitti. Toplam: {len(klima_links)} link.")
                        success = True
                    else:
                        print(f"🛑 Sayfa {p} boş! Captcha varsa doğrula, 5 sn bekleniyor...")
                        time.sleep(5)
                        attempts += 1
                except:
                    attempts += 1
                    time.sleep(2)
            current_page = p + 1
    except Exception as e:
        print(f"⚠️ Hata: {e}")
    
    driver.quit()
    print("🧹 RAM temizlendi, sonraki pakete geçiliyor...")

print(f"🏁 Klima Link Toplama Bitti! Toplam: {len(klima_links)} adet link hazır.")

🚀 Arçelik Klima linkleri toplanıyor. Hedef: 200 sayfa.
🌐 Yeni Oturum Başladı: Sayfa 1 - 20
✅ Sayfa 5 bitti. Toplam: 118 link.
✅ Sayfa 10 bitti. Toplam: 233 link.
✅ Sayfa 15 bitti. Toplam: 352 link.
✅ Sayfa 20 bitti. Toplam: 469 link.
🧹 RAM temizlendi, sonraki pakete geçiliyor...
🌐 Yeni Oturum Başladı: Sayfa 21 - 40
✅ Sayfa 25 bitti. Toplam: 584 link.
✅ Sayfa 30 bitti. Toplam: 696 link.
✅ Sayfa 35 bitti. Toplam: 810 link.
✅ Sayfa 40 bitti. Toplam: 927 link.
🧹 RAM temizlendi, sonraki pakete geçiliyor...
🌐 Yeni Oturum Başladı: Sayfa 41 - 60
✅ Sayfa 45 bitti. Toplam: 1043 link.
✅ Sayfa 50 bitti. Toplam: 1160 link.
✅ Sayfa 55 bitti. Toplam: 1275 link.
✅ Sayfa 60 bitti. Toplam: 1359 link.
🧹 RAM temizlendi, sonraki pakete geçiliyor...
🌐 Yeni Oturum Başladı: Sayfa 61 - 80
✅ Sayfa 65 bitti. Toplam: 1428 link.
✅ Sayfa 70 bitti. Toplam: 1496 link.
✅ Sayfa 75 bitti. Toplam: 1557 link.
✅ Sayfa 80 bitti. Toplam: 1609 link.
🧹 RAM temizlendi, sonraki pakete geçiliyor...
🌐 Yeni Oturum Başladı: Sayfa 81

In [3]:
session = requests.Session()
adapter = requests.adapters.HTTPAdapter(pool_connections=60, pool_maxsize=60)
session.mount('https://', adapter)

def fetch_klima_detail(item):
    url = item['url']
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    
    for attempt in range(4): # 4 kez deneme (Hata oranını düşüren kısım)
        try:
            time.sleep(random.uniform(0.1, 0.3) * (attempt + 1))
            res = session.get(url, headers=headers, timeout=12)
            
            if res.status_code == 200:
                soup = BeautifulSoup(res.text, 'html.parser')
                start_node = soup.find("div", {"data-ga-element": "Complaint_Content_Start"})
                content_parts = []
                
                if start_node:
                    for sibling in start_node.find_next_siblings():
                        if sibling.name == "div" and sibling.get("data-ga-element") == "Complaint_Content_End":
                            break
                        if sibling.name == "p":
                            content_parts.append(sibling.get_text(strip=True))
                
                full_text = " ".join(content_parts)
                if not full_text:
                    desc_div = soup.find("div", class_="complaint-detail-description")
                    full_text = desc_div.get_text(strip=True) if desc_div else ""

                if len(full_text) > 5:
                    return {"baslik": item["baslik"], "url": url, "aciklama": full_text}
            elif res.status_code == 429:
                time.sleep(8)
        except:
            continue
    return {"baslik": item["baslik"], "url": url, "aciklama": "HATA: Alınamadı."}

# --- ÇALIŞTIRICI ---
print(f"📥 {len(klima_links)} adet klima şikayeti çekiliyor...")
final_klima_data = []
start_time = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_klima_detail, link) for link in klima_links]
    for i, future in enumerate(futures, 1):
        final_klima_data.append(future.result())
        
        if i % 100 == 0:
            elapsed = time.time() - start_time
            rem = (elapsed / i) * (len(klima_links) - i) / 60
            print(f"📊 İlerleme: {i}/{len(klima_links)} | Kalan: ~{rem:.1f} dk")
            
        if i % 200 == 0: # Her 200 yorumda bir yedek kaydı
            pd.DataFrame(final_klima_data).to_excel("arcelik_klima_YEDEK.xlsx", index=False)



📥 2150 adet klima şikayeti çekiliyor...
📊 İlerleme: 100/2150 | Kalan: ~11.4 dk
📊 İlerleme: 200/2150 | Kalan: ~19.1 dk
📊 İlerleme: 300/2150 | Kalan: ~17.2 dk
📊 İlerleme: 400/2150 | Kalan: ~15.7 dk
📊 İlerleme: 500/2150 | Kalan: ~15.2 dk
📊 İlerleme: 600/2150 | Kalan: ~14.1 dk
📊 İlerleme: 700/2150 | Kalan: ~13.2 dk
📊 İlerleme: 800/2150 | Kalan: ~12.2 dk
📊 İlerleme: 900/2150 | Kalan: ~11.4 dk
📊 İlerleme: 1000/2150 | Kalan: ~10.4 dk
📊 İlerleme: 1100/2150 | Kalan: ~9.6 dk
📊 İlerleme: 1200/2150 | Kalan: ~8.7 dk
📊 İlerleme: 1300/2150 | Kalan: ~7.8 dk
📊 İlerleme: 1400/2150 | Kalan: ~6.8 dk
📊 İlerleme: 1500/2150 | Kalan: ~6.0 dk
📊 İlerleme: 1600/2150 | Kalan: ~5.1 dk
📊 İlerleme: 1700/2150 | Kalan: ~4.2 dk
📊 İlerleme: 1800/2150 | Kalan: ~3.3 dk
📊 İlerleme: 1900/2150 | Kalan: ~2.4 dk
📊 İlerleme: 2000/2150 | Kalan: ~1.4 dk
📊 İlerleme: 2100/2150 | Kalan: ~0.5 dk


In [4]:
# Final Kayıt
pd.DataFrame(final_klima_data).to_excel("arcelik_klima_final.xlsx", index=False)
print("✅ KLİMA TAMAMLANDI! Dosya: arcelik_klima_final.xlsx")

✅ KLİMA TAMAMLANDI! Dosya: arcelik_klima_final.xlsx
